In [ ]:
import pandas as pd
from PIL import Image
import numpy as np
import matplotlib.pyplot as plt
import keras
from sklearn.model_selection import train_test_split
from keras.layers import Conv2D, MaxPool2D, AveragePooling2D, Input, BatchNormalization, MaxPooling2D, Activation, Flatten, Dense, Dropout
from keras.models import Model
from keras.utils import to_categorical
from sklearn.metrics import classification_report
from imblearn.over_sampling import RandomOverSampler
from keras.preprocessing import image
import scipy
import os
import cv2

In [ ]:
df = pd.read_csv('/content/drive/MyDrive/train_emotions.csv')

In [ ]:
df.shape

(28709, 2)

In [ ]:
df.columns.values

array(['emotion', 'pixels'], dtype=object)

In [ ]:
count_df = df.groupby('emotion').count()
count_df

,pixels
emotion,
0,3995
1,436
2,4097
3,7215
4,4965
5,4830
6,3171


In [ ]:
frames = []
for emo in range(7):
  sliced_df = df[df['emotion']==emo]
  if emo==1:
    frames.append(sliced_df)
  else:
    frames.append(sliced_df.head(2000))

df = pd.concat(frames)

In [ ]:
df.groupby('emotion').count()

,pixels
emotion,
0,2000
1,436
2,2000
3,2000
4,2000
5,2000
6,2000


In [ ]:
pixel_data = df['pixels']
label_data = df['emotion']

In [ ]:
def preprocess_pixels(pixel_data):
  images = []
  for i in range(len(pixel_data)):
    x = pixel_data[i]
    x = x.replace('[', '')
    x = x.replace(']', '')
    img = np.fromstring(x, dtype='float32', sep=', ')
    img = img.reshape(48,48,1)
    images.append(img)

  X = np.array(images)
  
 

  return X

In [ ]:
oversampler = RandomOverSampler(sampling_strategy='auto')

X_over, Y_over = oversampler.fit_resample(pixel_data.values.reshape(-1,1), label_data)

X_over_series = pd.Series(X_over.flatten())

In [ ]:
X = preprocess_pixels(X_over_series)
Y = np.array(Y_over).astype('float32')
Y = Y.reshape(Y.shape[0],1)

X_train, X_test, Y_train, Y_test = train_test_split(X, Y, test_size = 0.2, random_state = 9)

In [ ]:
Y_train.shape

(11200, 1)

In [ ]:
def emotion_recognition(input_shape):

  X_input = Input(input_shape)

  X = Conv2D(32, kernel_size=(3,3), strides=(1,1), padding='valid')(X_input)
  X = BatchNormalization(axis=3)(X)
  X = Activation('relu')(X)


  X = Conv2D(64, (3,3), strides=(1,1), padding = 'same')(X)
  X = BatchNormalization(axis=3)(X)
  X = Activation('relu')(X)

  X = MaxPooling2D((2,2))(X)

  X = Conv2D(64, (3,3), strides=(1,1), padding = 'valid')(X)
  X = BatchNormalization(axis=3)(X)
  X = Activation('relu')(X)

  X = Conv2D(128, (3,3), strides=(1,1), padding = 'same')(X)
  X = BatchNormalization(axis=3)(X)
  X = Activation('relu')(X)


  X = MaxPooling2D((2,2))(X)

  X = Conv2D(128, (3,3), strides=(1,1), padding = 'valid')(X)
  X = BatchNormalization(axis=3)(X)
  X = Activation('relu')(X)

 

  X = MaxPooling2D((2,2))(X)
  X = Flatten()(X)
  X = Dense(200, activation='relu')(X)
  X = Dropout(0.6)(X)
  X = Dense(7, activation = 'softmax')(X)

  model = Model(inputs=X_input, outputs=X)

  return model

In [ ]:
model = emotion_recognition((48,48,1))
adam = keras.optimizers.Adam(learning_rate=0.001)
model.compile(optimizer=adam, loss='categorical_crossentropy', metrics=['accuracy'])

In [ ]:
y_train = to_categorical(Y_train, num_classes=7)

y_test = to_categorical(Y_test, num_classes=7)

In [ ]:
model.fit(X_train, y_train, epochs=30, validation_data=(X_test, y_test))

Epoch 1/30
350/350 [==============================] - 275s 784ms/step - loss: 1.9749 - accuracy: 0.1676 - val_loss: 1.9072 - val_accuracy: 0.2454
Epoch 2/30
350/350 [==============================] - 274s 783ms/step - loss: 1.8898 - accuracy: 0.1928 - val_loss: 1.7785 - val_accuracy: 0.2746
Epoch 3/30
350/350 [==============================] - 271s 774ms/step - loss: 1.8574 - accuracy: 0.2170 - val_loss: 1.7905 - val_accuracy: 0.3396
Epoch 4/30
350/350 [==============================] - 269s 770ms/step - loss: 1.8387 - accuracy: 0.2260 - val_loss: 1.7538 - val_accuracy: 0.3236
Epoch 5/30
350/350 [==============================] - 270s 771ms/step - loss: 1.8188 - accuracy: 0.2462 - val_loss: 1.7123 - val_accuracy: 0.3668
Epoch 6/30
350/350 [==============================] - 269s 769ms/step - loss: 1.7892 - accuracy: 0.2646 - val_loss: 1.6630 - val_accuracy: 0.3636
Epoch 7/30
350/350 [==============================] - 272s 776ms/step - loss: 1.7712 - accuracy: 0.2748 - val_loss: 1.6952 -

In [ ]:
# model_json = model.to_json()
# with open("model.json", "w") as json_file:
#   json_file.write(model_json)
#   model.save_weights("model.h5")
#   print("Saved model to disk")

Saved model to disk


In [ ]:
# from google.colab import files
# files.download('model.h5')

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

In [ ]:
preds = model.predict(X_train)
def get_class(preds):
  pred_class = np.zeros((preds.shape[0],1))

  for i in range(len(preds)):
   pred_class[i] = np.argmax(preds[i])

  return pred_class

pred_class_train = get_class(preds)

350/350 [==============================] - 57s 164ms/step


In [ ]:
label_dict = {0 : 'Angry', 1 : 'Disgust', 2 : 'Fear', 3 : 'Happiness', 5 : 'Sad', 6 : 'Surprise', 4 : 'Neutral'}

In [ ]:
from google.colab import files
uploaded = files.upload()

Saving im43.png to im43.png


In [ ]:
import tensorflow as tf

In [ ]:
def predict_emo(label_dict, img_path):
  # img_path = 'im43.png'
  # img = image.load_img(img_path, grayscale=True, target_size=(48,48))

  img = tf.keras.utils.load_img(img_path, grayscale=True, target_size=(48,48))


  x = tf.keras.utils.img_to_array(img)
  x = np.expand_dims(x, axis=0)

  prediction = np.argmax(model.predict(x))
  return label_dict[prediction]
  # print('The predicted emotion is : ' + label_dict[prediction])
  # my_image = tf.keras.utils.load_img(img_path)
  # plt.imshow(my_image)

In [ ]:
results = []
for idx in range(100):
  file_path = '/content/drive/MyDrive/angry/im'+str(idx)+'.png'
  r = predict_emo(label_dict, file_path)
  results.append(r)

ctr = [1]*len(results)

data = {'emo': results, 'ctr': ctr}
pd.DataFrame(data).groupby('emo').count()

In [ ]:
xxx = {'just':[1,2,3,4],'like':[1,1,1,1]}
pd.DataFrame(xxx).groupby('just').count()

,like
just,
1,1
2,1
3,1
4,1
